In [ ]:
from transformers import pipeline
from datasets import load_dataset
from transformers import AutoTokenizer


model_pipeline = pipeline(
    model = "Qwen/Qwen2.5-3B-Instruct",
    # device = "cuda"
)


print(model_pipeline("What is Truvada?"))

c:\Users\munod\OneDrive\Documents\Projects\auto_surgical_report\surg_venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
c:\Users\munod\OneDrive\Documents\Projects\auto_surgical_report\surg_venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symli

[{'generated_text': 'What is Truvada? Truvada (tenofovir disoproxil fumarate and emtricitabine) is a combination antiretroviral medication used to prevent HIV infection in people who are at high risk of exposure. It is approved for use in adults and adolescents aged 12 years or older.\nHow does Truvada work? Truvada works by preventing the virus from multiplying in the body. It contains two active ingredients, tenofovir and emtricitabine, which work together to block the virus from infecting new cells.\nWhat are the potential side effects of Truvada? The most common side effects of Truvada include nausea, headache, diarrhea, and abdominal pain. Less common side effects may include liver problems, kidney damage, and bone loss. In rare cases, Truvada has been associated with an increased risk of kidney stones and pancreatitis.\nCan Truvada be taken with other medications? Truvada can be taken with other medications, but it is important to consult with a healthcare provider before startin

In [2]:
print(model_pipeline("What is Truvada?")[0]['generated_text'])

What is Truvada? Truvada is a drug that contains two antiretroviral medications, emtricitabine and tenofovir disoproxil fumarate. It can be used to prevent HIV infection in people who are at high risk for contracting the virus. Truvada may also be used to treat people with HIV.
How does Truvada work? Truvada works by preventing the human immunodeficiency virus (HIV) from multiplying in the body. When HIV infects an individual, it attaches to immune cells and begins to reproduce. As it reproduces, the virus damages or kills the infected cells. Over time, this process weakens the immune system, making the person more susceptible to illnesses.
Truvada prevents HIV from entering immune cells. Once inside, the virus can no longer multiply. This reduces the amount of virus in the blood and lowers the chance of transmitting the virus to others.
Is Truvada safe? Yes, Truvada has been shown to be safe and effective when used as directed. However, like all medications, Truvada can have side effe

In [ ]:


tokenizer = AutoTokenizer.from_pretrained(
    "Qwen/Qwen2.5-3B-Instruct"
)

In [ ]:


train_data = load_dataset("json", data_files="training_data.json")

Generating train split: 2 examples [00:00, 54.96 examples/s]


In [4]:
train_data

DatasetDict({
    train: Dataset({
        features: ['prompt', 'completion'],
        num_rows: 2
    })
})